In [ ]:
from smooth import is_smooth
from random import randint, seed, random, Random

In [ ]:
_rng = Random()

def rand_smooth(y, B):
    x = 1
    while not is_smooth(y, x):
        x = _rng.randrange(1, B + 1)
    return x